In [ ]:
"""сокращаем импорт"""

from __future__ import division, print_function

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

"""дополнительный блок на решение прикладных задач"""

import datetime
import datetime as dt
from collections import Counter

"""без графиков нам больше не нужно"""

from sklearn.ensemble.forest import RandomForestRegressor

pd.set_option('display.max_columns', 64)

"""на предупреждение снизу внимание обращать не надо"""

from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from scipy.optimize import minimize

from sklearn.model_selection import TimeSeriesSplit

from statsmodels.iolib.table import SimpleTable
import tables

import matplotlib.pyplot as plt

from datetime import timedelta

# 13_08_2020 работа с экспонентами

from numpy import inf

In [ ]:
# текущая версия автоматизации, уехала из основного алгоритма из за нагрузки на кеш. 
# питаеться файлами, которые получаються в выводе сис.сквозной аналитики:no_life[n]
# позволяет смотреть препроцессинг глазками не залезая в дебри основной сиситемы

def eva_00():
    
    """      
    фукнция путей
    
    """     
    
    result_utt = pd.read_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/utt_math.csv', sep=',', encoding="utf-8") 
    
    result_sam= pd.read_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/sam_math.csv', sep=',', encoding="utf-8") 
    
    result_mtk= pd.read_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtk_meath.csv', sep=',', encoding="utf-8") 
    
    result_mtc = pd.read_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtc_math.csv', sep=',', encoding="utf-8") 
    
    result_cher= pd.read_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/cher_math.csv', sep=',', encoding="utf-8") 
    
    result_nvds = pd.read_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nvds_math.csv', sep=',', encoding="utf-8")    
    
    result_trans = pd.read_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/trans_math.csv', sep=',', encoding="utf-8") 
        
    result_nst = pd.read_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nst_math.csv', sep=',', encoding="utf-8")   
    
    def soft_no_life(df):
        """потому что no_life[n] очень нежный, как моя израненая душа и он мутирует на записи\чтении"""
        
        df['delta'] =  pd.to_timedelta(df['delta']) 
        df['простой'] =  pd.to_timedelta(df['простой']) 
        datetime_index = pd.DatetimeIndex(df._Дата.values)
        df = df.set_index(datetime_index)  
        
        return df
   
    result_utt = soft_no_life(result_utt)
    result_nst = soft_no_life(result_nst)
    result_trans = soft_no_life(result_trans)
    result_nvds = soft_no_life(result_nvds)
    result_cher = soft_no_life(result_cher)
    result_mtc = soft_no_life(result_mtc)
    result_mtk = soft_no_life(result_mtk)
    result_sam = soft_no_life(result_sam)    
    
    # result_utt,result_nst,result_trans,result_nvds,result_cher,result_mtc,result_mtk,result_sam = eva_00()
    
    return result_utt, result_nst,result_trans,result_nvds,result_cher,result_mtc,result_mtk,result_sam

In [ ]:
def eva_01():
    """
    функция хранимого генератора
      
    """
    
    # result_utt c доработками от 06_08_2020
    
    result_utt,result_nst,result_trans,result_nvds,result_cher,result_mtc,result_mtk,result_sam = eva_00()
    
    # 21.10.2020 вероятнее всего во всех витринах необходима колонка с номером под result
    
    result_utt['Номер'] = result_utt['_ГарНомер']
    
    result_utt['_ГарНомер'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]', value=r'')   
    
    # 26_10_2020 добавлена колонка ['номер'] в остальные таблицы
    
    result_nst['Номер'] = result_nst['_ГарНомер']
    result_trans['Номер'] = result_trans['_ГарНомер']
    result_nvds['Номер'] = result_nvds['_ГарНомер']
    result_cher['Номер'] = result_cher['_ГарНомер']
    result_mtc['Номер'] = result_mtc['_ГарНомер']
    result_mtk['Номер'] = result_mtk['_ГарНомер']
    result_sam['Номер'] = result_sam['_ГарНомер']
    
    def super_test(df):
        df['_ГарНомер'] = df['_ГарНомер'].astype('int64')
        return df
    
    result_utt = super_test(result_utt)
    result_nst = super_test(result_nst)
    result_trans= super_test(result_trans)
    result_nvds = super_test(result_nvds)
    result_cher = super_test(result_cher)
    result_mtc = super_test(result_mtc)
    result_mtk = super_test(result_mtk)
    result_sam = super_test(result_sam)      

    return (result_utt,result_nst,result_trans,result_nvds,
            result_cher,result_mtc,result_mtk,result_sam)

In [ ]:
def anime_holt(x,df_n):
    
    name  = x
    
    df = df_n[df_n['_ГарНомер']==name]
    
    print(name)

    class HoltWinters:
    
        """
        Модель Хольта-Винтерса 
    
        # series - исходный временной ряд
        # slen - длина сезона
        # alpha, beta, gamma - коэффициенты модели Хольта-Винтерса
        # n_preds - горизонт предсказаний
        # scaling_factor - задаёт ширину доверительного интервала по Брутлагу (обычно принимает значения от 2 до 3)
    
        """
    
        def __init__(self, series, slen, alpha, beta, gamma, n_preds, scaling_factor=1.96):
            self.series = series
            self.slen = slen
            self.alpha = alpha
            self.beta = beta
            self.gamma = gamma
            self.n_preds = n_preds
            self.scaling_factor = scaling_factor
    
        def initial_trend(self):
            sum = 0.0
            for i in range(self.slen):
                sum += float(self.series[i+self.slen] - self.series[i]) / self.slen
            return sum / self.slen  
    
        def initial_seasonal_components(self):
            seasonals = {}
            season_averages = []
            n_seasons = int(len(self.series)/self.slen)
            # вычисляем сезонные средние
            for j in range(n_seasons):
                season_averages.append(sum(self.series[self.slen*j:self.slen*j+self.slen])/float(self.slen))
            # вычисляем начальные значения
            for i in range(self.slen):
                sum_of_vals_over_avg = 0.0
                for j in range(n_seasons):
                    sum_of_vals_over_avg += self.series[self.slen*j+i]-season_averages[j]
                seasonals[i] = sum_of_vals_over_avg/n_seasons
            return seasonals   
    
        def triple_exponential_smoothing(self):
            self.result = []
            self.Smooth = []
            self.Season = []
            self.Trend = []
            self.PredictedDeviation = []
            self.UpperBond = []
            self.LowerBond = []
    
            seasonals = self.initial_seasonal_components()
    
            for i in range(len(self.series)+self.n_preds):
                if i == 0: # инициализируем значения компонент
                    smooth = self.series[0]
                    trend = self.initial_trend()
                    self.result.append(self.series[0])
                    self.Smooth.append(smooth)
                    self.Trend.append(trend)
                    self.Season.append(seasonals[i%self.slen])
    
                    self.PredictedDeviation.append(0)
    
                    self.UpperBond.append(self.result[0] + 
                                          self.scaling_factor * 
                                          self.PredictedDeviation[0])
    
                    self.LowerBond.append(self.result[0] - 
                                          self.scaling_factor * 
                                          self.PredictedDeviation[0])
    
                    continue
                    
                if i >= len(self.series): # прогнозируем
                    m = i - len(self.series) + 1
                    self.result.append((smooth + m*trend) + seasonals[i%self.slen])
    
                    # во время прогноза с каждым шагом увеличиваем неопределенность
                    self.PredictedDeviation.append(self.PredictedDeviation[-1]*1.01) 
    
                else:
                    val = self.series[i]
                    last_smooth, smooth = smooth, self.alpha*(val-seasonals[i%self.slen]) + (1-self.alpha)*(smooth+trend)
                    trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                    seasonals[i%self.slen] = self.gamma*(val-smooth) + (1-self.gamma)*seasonals[i%self.slen]
                    self.result.append(smooth+trend+seasonals[i%self.slen])
    
                    # Отклонение рассчитывается в соответствии с алгоритмом Брутлага
                    self.PredictedDeviation.append(self.gamma * np.abs(self.series[i] - self.result[i]) 
                                                   + (1-self.gamma)*self.PredictedDeviation[-1])
    
                self.UpperBond.append(self.result[-1] + 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[-1])
    
                self.LowerBond.append(self.result[-1] - 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[-1])
    
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i % self.slen])
    
    # размечаем на выходные и рабочие дни
    
    df["weekday"] = df.index.weekday   
    df['is_weekend'] = df.weekday.isin([5,6])*1
    
    def code_mean(data, cat_feature, real_feature):
        """
        Возвращает словарь, где ключами являются уникальные категории признака cat_feature, 
        а значениями - средние по real_feature
        """
        return dict(data.groupby(cat_feature)[real_feature].mean())
    
    mean_all = code_mean(df, 'weekday', "math")
    
    # средние значения по дистанции до следующего листа
    
    mean_weekday_work = code_mean(df, 'weekday', "math")
    mean_weekend_work = code_mean(df, 'is_weekend', "math")
    
    # для code_mean wait
    
    df['wait'] = df.delta.copy()
    
    # df.wait = df.wait.dt.days.astype('int16')
    
    try:
        df.wait = df.wait.dt.days.astype('int16')
    except ValueError:
        df.wait = df.wait.fillna(df.delta.mean())
        df.wait = df.wait.dt.days.astype('int16')
        
    # это не бага! это БАЖИЩЕ 19_11_2020
    
    df = df[df.range != 0]    
    
    # средние значения по времени ремонтных работ
    
    mean_weekday_wait =  code_mean(df, 'weekday', "wait")
    mean_weekend_wait =  code_mean(df, 'is_weekend', "wait")
    
    # test_math передаём Хольту
    
    test_math = df.drop(['_ГарНомер', 'delta', 'range', 'класс', 'активность',
           'надёжность', 'простой', 'wait','weekday','is_weekend',"_Дата"], axis=1)
    
    # нужно для логарифмирования, 2019-02-23 till 2019-02-23 gives INF (zero) in log we want
    
    test_math_log = test_math.loc[~(test_math==0).all(axis=1)]
    
    test_math_log['log_value'] = np.log(test_math_log['math'])
        
    # test_wait отдадим Винтерсу
    
    test_wait = df.drop(['_ГарНомер', 'delta', 'range', 'класс', 'активность',
           'надёжность', 'простой', 'math','weekday','is_weekend',"_Дата"], axis=1)
    
    # 4 модели - 2 логарифмированные, две нет
    
    test_wait_log = test_wait.loc[~(test_wait==0).all(axis=1)]
    
    test_wait_log['log_value'] = np.log(test_wait_log['wait'])
    
    # две недели коту под хвост
    
    test_wait_log = test_wait_log.replace(-inf, 0.0)
    
    # запоминаем статистику
    
    test_math_work_mean = test_math.math.mean()
    test_math_work_median = test_math.math.median()
    
    test_math_work_mean_log = test_math_log.math.mean()
    test_math_work_median_log = test_math_log.math.median()
    
    test_math_wait_mean = test_wait.wait.mean()
    test_math_wait_median = test_wait.wait.median()
    
    test_math_wait_mean_log = test_wait_log.wait.mean()
    test_math_wait_median_log = test_wait_log.wait.median()
    
    # определяем размер данных для тестирования ( тестиируем до Т% данных с начала)
    
    time_1 = int(len(test_math)/100*25)
    time_2 = int(len(test_math)/100*50) 
    time_3 = int(len(test_math)/100*75) 
    time_4 = len(test_math) 
    
                 
    time_5 = int(len(test_math_log)/100*25)
    time_6 = int(len(test_math_log)/100*50) 
    time_7 = int(len(test_math_log)/100*75) 
    time_8 = len(test_math_log) 
    
    time_9 = int(len(test_wait )/100*25)
    time_10 = int(len(test_wait )/100*50)
    time_11 = int(len(test_wait )/100*75)
    time_12 = len(test_wait)
    
    time_13 = int(len(test_wait_log )/100*25)
    time_14 = int(len(test_wait_log )/100*50)
    time_15 = int(len(test_wait_log )/100*75)
    time_16 = len(test_wait)
    
    #костыли кросс валидации - суровы как злые некко, вместо  test_math.math[:time_1].values должно быть
    # values = name.values где name def func_name(name): но конкретно в этот костыль пихнуть name нельзя
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_math.math[:time_1].values
        alpha, beta, gamma = x
    
        # задаём число фолдов для кросс-валидации
        
        tscv = TimeSeriesSplit(n_splits=3) 
        
        # идем по фолдам, на каждом обучаем модель, строим прогноз на отложенной выборке и считаем ошибку
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)
            
        # Возвращаем средний квадрат ошибки по вектору ошибок, кроссвалидация учиться на нём 
        return np.mean(np.array(errors))
    
    # то что дёргает класс и кросс валидацию выше, будем ддёргать 16 раз ради 4х моделей
    # при этом нам придёться 16 раз написать timseriesCVscore поскольку она не поддерживает передачу переменных внутрь.
    
    # инициализируем значения параметров
    
    x = [0, 0, 0] 
    
    """
    
    # вне перехвата исключений проблемы симметрии - выбора
    
    # Минимизируем функцию потерь с ограничениями на параметры
    
    opt_math1 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
    
    # Из оптимизатора берем оптимальное значение параметров
    
    alpha_final_m1, beta_final_m1, gamma_final_m1 = opt_math1.x
    
    option_m1 = alpha_final_m1, beta_final_m1, gamma_final_m1
    
    print("opt_math_1 : ",alpha_final_m1, beta_final_m1, gamma_final_m1)
    
    """
    
    try:
        opt_math1 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_m1, beta_final_m1, gamma_final_m1 = opt_math1.x
        option_m1 = alpha_final_m1, beta_final_m1, gamma_final_m1
    except IndexError:
        opt_math1 = None
        alpha_final_m1, beta_final_m1, gamma_final_m1 = (1,1,1)
        option_m1 = alpha_final_m1, beta_final_m1, gamma_final_m1
    except ValueError:
        opt_math1 = None
        alpha_final_m1, beta_final_m1, gamma_final_m1 = (1,1,1)
        option_m1 = alpha_final_m1, beta_final_m1, gamma_final_m1
    
    print("opt_math_1 : ",alpha_final_m1, beta_final_m1, gamma_final_m1)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_math.math[:time_2].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)        
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
    
    try:
        opt_math2 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_m2, beta_final_m2, gamma_final_m2 = opt_math2.x
        option_m2 = alpha_final_m2, beta_final_m2, gamma_final_m2
    except IndexError:
        opt_math2 = None
        alpha_final_m2, beta_final_m2, gamma_final_m2 = (1,1,1)
        option_m2 = alpha_final_m2, beta_final_m2, gamma_final_m2
    except ValueError:
        opt_math2 = None
        alpha_final_m2, beta_final_m2, gamma_final_m2 = (1,1,1)
        option_m2 = alpha_final_m2, beta_final_m2, gamma_final_m2
    
    print("opt_math_2 : ",alpha_final_m2, beta_final_m2, gamma_final_m2)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_math.math[:time_3].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)        
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
          
    try:
        opt_math3 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_m3, beta_final_m3, gamma_final_m3 = opt_math3.x
        option_m3 = alpha_final_m3, beta_final_m3, gamma_final_m3
    except IndexError:
        opt_math3 = None
        alpha_final_m3, beta_final_m3, gamma_final_m3 = (1,1,1)
        option_m3 = alpha_final_m3, beta_final_m3, gamma_final_m3
    except ValueError:
        opt_math3 = None
        alpha_final_m3, beta_final_m3, gamma_final_m3 = (1,1,1)
        option_m3 = alpha_final_m3, beta_final_m3, gamma_final_m3
          
    print("opt_math_3 : ",alpha_final_m3, beta_final_m3, gamma_final_m3)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_math.math[:time_4].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)        
        
        return np.mean(np.array(errors))
          
    x = [0,0,0]
          
    try:
        opt_math4 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_m4, beta_final_m4, gamma_final_m4 = opt_math4.x
        option_m4 = alpha_final_m4, beta_final_m4, gamma_final_m4
    except IndexError:
        opt_math4 = None
        alpha_final_m4, beta_final_m4, gamma_final_m4 = (1,1,1)
        option_m4 = alpha_final_m4, beta_final_m4, gamma_final_m4
    except ValueError:
        opt_math4 = None
        alpha_final_m4, beta_final_m4, gamma_final_m4 = (1,1,1)
        option_m4 = alpha_final_m4, beta_final_m4, gamma_final_m4
          
    print("opt_math_4 : ",alpha_final_m4, beta_final_m4, gamma_final_m4)
    
    # а теперь тоже самое с логарифмированием
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_math_log.log_value[:time_5].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)
            
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
    
    try:
        opt_math5 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_ml1, beta_final_ml1, gamma_final_ml1 = opt_math5.x
        option_m5 = alpha_final_ml1, beta_final_ml1, gamma_final_ml1
    except IndexError:
        opt_math5 = None
        alpha_final_ml1, beta_final_ml1, gamma_final_ml1 = (1,1,1)
        option_m5 = alpha_final_ml1, beta_final_ml1, gamma_final_ml1
    except ValueError:
        opt_math5 = None
        alpha_final_ml1, beta_final_ml1, gamma_final_ml1 = (1,1,1)
        option_m5 = alpha_final_ml1, beta_final_ml1, gamma_final_ml1
    
    print("opt_math_ml_1 : ",alpha_final_ml1, beta_final_ml1, gamma_final_ml1)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_math_log.log_value[:time_6].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)        
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
    
    try:
        opt_math6 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_ml2, beta_final_ml2, gamma_final_ml2 = opt_math6.x
        option_m6 = alpha_final_ml2, beta_final_ml2, gamma_final_ml2
    except IndexError:
        opt_math6 = None
        alpha_final_ml2, beta_final_ml2, gamma_final_ml2 = (1,1,1)
        option_m6 = alpha_final_ml2, beta_final_ml2, gamma_final_ml2
    except ValueError:
        opt_math6 = None
        alpha_final_ml2, beta_final_ml2, gamma_final_ml2 = (1,1,1)
        option_m6 = alpha_final_ml2, beta_final_ml2, gamma_final_ml2
    
    print("opt_math_ml_2 : ",alpha_final_ml2, beta_final_ml2, gamma_final_ml2)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_math_log.log_value[:time_7].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)
            
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
    
    try:
        opt_math7 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_ml3, beta_final_ml3, gamma_final_ml3 = opt_math7.x
        option_m7 = alpha_final_ml3, beta_final_ml3, gamma_final_ml3
    except IndexError:
        opt_math7 = None
        alpha_final_ml3, beta_final_ml3, gamma_final_ml3 = (1,1,1)
        option_m7 = alpha_final_ml3, beta_final_ml3, gamma_final_ml3
    except ValueError:
        opt_math7 = None
        alpha_final_ml3, beta_final_ml3, gamma_final_ml3 = (1,1,1)
        option_m7 = alpha_final_ml3, beta_final_ml3, gamma_final_ml3
    
    print("opt_math_ml_3 : ",alpha_final_ml3, beta_final_ml3, gamma_final_ml3)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_math_log.log_value[:time_8].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)        
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
          
    try:
        opt_math8 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_ml4, beta_final_ml4, gamma_final_ml4 = opt_math8.x
        option_m8 = alpha_final_ml2, beta_final_ml2, gamma_final_ml2
    except IndexError:
        opt_math8 = None
        alpha_final_ml4, beta_final_ml4, gamma_final_ml4 = (1,1,1)
        option_m8 = alpha_final_ml4, beta_final_ml4, gamma_final_ml4
    except ValueError:
        opt_math8 = None
        alpha_final_ml4, beta_final_ml4, gamma_final_ml4 = (1,1,1)
        option_m8 = alpha_final_ml4, beta_final_ml4, gamma_final_ml4
          
    print("opt_math_ml_4 : ",alpha_final_ml4, beta_final_ml4, gamma_final_ml4)
    
    # а теперь простой
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_wait.wait[:time_9].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)        
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
          
    try:
        opt_math9 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_w1, beta_final_w1, gamma_final_w1 = opt_math9.x
        option_w1 = alpha_final_w1, beta_final_w1, gamma_final_w1
    except IndexError:
        opt_math9 = None
        alpha_final_w1, beta_final_w1, gamma_final_w1 = (1,1,1)
        option_w1 = alpha_final_w1, beta_final_w1, gamma_final_w1
    except ValueError:
        opt_math9 = None
        alpha_final_w1, beta_final_w1, gamma_final_w1 = (1,1,1)
        option_w1 = alpha_final_w1, beta_final_w1, gamma_final_w1
    
    print("opt_wait_1 : ",alpha_final_w1, beta_final_w1, gamma_final_w1)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_wait.wait[:time_10].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)
            
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
    
    try:
        opt_math10 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_w2, beta_final_w2, gamma_final_w2 = opt_math10.x
        option_w2 = alpha_final_w2, beta_final_w2, gamma_final_w2
    except IndexError:
        opt_math10 = None
        alpha_final_w2, beta_final_w2, gamma_final_w2 = (1,1,1)
        option_w2 = alpha_final_w2, beta_final_w2, gamma_final_w2
    except ValueError:
        opt_math9 = None
        alpha_final_w2, beta_final_w2, gamma_final_w2 = (1,1,1)
        option_w2 = alpha_final_w2, beta_final_w2, gamma_final_w2
    
    print("opt_wait_2 : ",alpha_final_w2, beta_final_w2, gamma_final_w2)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_wait.wait[:time_11].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)        
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
    
    try:
        opt_math11 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_w3, beta_final_w3, gamma_final_w3 = opt_math11.x
        option_w3 = alpha_final_w3, beta_final_w3, gamma_final_w3
    except IndexError:
        opt_math11 = None
        alpha_final_w3, beta_final_w3, gamma_final_w3 = (1,1,1)
        option_w3 = alpha_final_w3, beta_final_w3, gamma_final_w3
    except ValueError:
        opt_math11 = None
        alpha_final_w3, beta_final_w3, gamma_final_w3 = (1,1,1)
        option_w3 = alpha_final_w3, beta_final_w3, gamma_final_w3
    
    print("opt_wait_3 : ",alpha_final_w3, beta_final_w3, gamma_final_w3)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_wait.wait[:time_12].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)
            
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
    
    try:
        opt_math12 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_w4, beta_final_w4, gamma_final_w4 = opt_math12.x
        option_w4 = alpha_final_w4, beta_final_w4, gamma_final_w4
    except IndexError:
        opt_math12 = None
        alpha_final_w4, beta_final_w4, gamma_final_w4 = (1,1,1)
        option_w4 = alpha_final_w4, beta_final_w4, gamma_final_w4
    except ValueError:
        opt_math12 = None
        alpha_final_w4, beta_final_w4, gamma_final_w4 = (1,1,1)
        option_w4 = alpha_final_w4, beta_final_w4, gamma_final_w4
    
    print("opt_wait_4 : ",alpha_final_w4, beta_final_w4, gamma_final_w4)
    
    # а теперь логарифмируем простой и поглядим что там будет, заита от глупостей = 1.1.1
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_wait_log.log_value[:time_13].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)
            
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
    
    try:
        opt_math13 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_wl1, beta_final_wl1, gamma_final_wl1 = opt_math13.x
        option_w5 = alpha_final_wl1, beta_final_wl1, gamma_final_wl1
    except IndexError:
        opt_math13 = None
        alpha_final_wl1, beta_final_wl1, gamma_final_wl1 = (1,1,1)
        option_w5 = alpha_final_wl1, beta_final_wl1, gamma_final_wl1
    except ValueError:
        opt_math13 = None
        alpha_final_wl1, beta_final_wl1, gamma_final_wl1 = (1,1,1)
        option_w5 = alpha_final_wl1, beta_final_wl1, gamma_final_wl1
    
    print("opt_wait_log_1 : ",alpha_final_wl1, beta_final_wl1, gamma_final_wl1)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_wait_log.log_value[:time_14].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)
            
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
          
    try:
        opt_math14 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_wl2, beta_final_wl2, gamma_final_wl2 = opt_math14.x
        option_w6 = alpha_final_wl2, beta_final_wl2, gamma_final_wl2
    except IndexError:
        opt_math14 = None
        alpha_final_wl2, beta_final_wl2, gamma_final_wl2 = (1,1,1)
        option_w6 = alpha_final_wl2, beta_final_wl2, gamma_final_wl2
    except ValueError:
        opt_math6 = None
        alpha_final_wl2, beta_final_wl2, gamma_final_wl2 = (1,1,1)
        option_w6 = alpha_final_wl2, beta_final_wl2, gamma_final_wl2
    
    print("opt_wait_log_2 : ",alpha_final_wl2, beta_final_wl2, gamma_final_wl2)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_wait_log.log_value[:time_15].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)        
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
       
    try:
        opt_math15 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_wl3, beta_final_wl3, gamma_final_wl3 = opt_math15.x
        option_w7 = alpha_final_wl3, beta_final_wl3, gamma_final_wl3
    except IndexError:
        opt_math15 = None
        alpha_final_wl3, beta_final_wl3, gamma_final_wl3 = (1,1,1)
        option_w7 = alpha_final_wl3, beta_final_wl3, gamma_final_wl3
    except ValueError:
        opt_math15 = None
        alpha_final_wl3, beta_final_wl3, gamma_final_wl3 = (1,1,1)
        option_w7 = alpha_final_wl3, beta_final_wl3, gamma_final_wl3  
          
    print("opt_wait_log_3 : ",alpha_final_wl3, beta_final_wl3, gamma_final_wl3)
    
    def timeseriesCVscore(x):
        # вектор ошибок
        errors = []
    
        values = test_wait_log.log_value[:time_16].values
        alpha, beta, gamma = x
        tscv = TimeSeriesSplit(n_splits=3) 
        
        
        for train, test in tscv.split(values):
    
            model = HoltWinters(series=values[train], slen = 1, # длинна сезона
                                alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
            model.triple_exponential_smoothing()
    
            predictions = model.result[-len(test):]
            actual = values[test]
            error = mean_squared_error(predictions, actual)
            errors.append(error)
            
        
        return np.mean(np.array(errors))
    
    x = [0, 0, 0] 
    
    # 20_07_2020 нашёл ошибку в alpha_final_w4, beta_final_wl4, gamma_final_wl4 = (1,1,1)
          
    try:
        opt_math16 = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))
        alpha_final_wl4, beta_final_wl4, gamma_final_wl4 = opt_math16.x
        option_w8 = alpha_final_wl4, beta_final_wl4, gamma_final_wl4
    except IndexError:
        opt_math16 = None
        alpha_final_wl4, beta_final_wl4, gamma_final_wl4 = (1,1,1)
        option_w8 = alpha_final_wl4, beta_final_wl4, gamma_final_wl4
    except ValueError:
        opt_math16 = None
        alpha_final_wl4, beta_final_wl4, gamma_final_wl4 = (1,1,1)
        option_w8 = alpha_final_wl4, beta_final_wl4, gamma_final_wl4
    
    print("opt_wait_log_4 : ",alpha_final_wl4, beta_final_wl4, gamma_final_wl4, )
    
    # выбираем лучший отрезок по которому выберем лучшие параметры.
    
    math_roll = {'Models':[option_m1, option_m2, option_m3, option_m4,
                            option_m5,option_m6,option_m7,option_m8],
            'Periods':[time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8],
                'Types':[1,1,1,1,2,2,2,2]}
    
    math_roll = pd.DataFrame(math_roll)
    
    
    wait_roll = {'Models':[option_w1, option_w2, option_w3, option_w4,
                            option_w5,option_w6,option_w7,option_w8],             
                 'Periods':[time_9,time_10,time_11,time_12,time_13,time_14,time_15,time_16],
                'Types':[1,1,1,1,2,2,2,2]}
    
    wait_roll = pd.DataFrame(wait_roll)
    
    # интерпретация результатов тоже сложная, няша внимание!
    
    # 1 сбрасываем нули в math_roll и wait_roll я в них не верю.
    
    # собака зарыта здесь
    
    math_roll = math_roll.loc[math_roll['Models'] != (0,0,0)]
    math_roll = math_roll.loc[math_roll['Models'] != (1,1,1)]
    
    wait_roll = wait_roll.loc[wait_roll['Models'] != (0,0,0)]
    wait_roll = wait_roll.loc[wait_roll['Models'] != (1,1,1)]
    
    # если в выборке результатов оценки будут только 1,1,1 - тогда варитивность будет равно 1
    # 1,1,1 - это значение среднеквадратичной ошибки, это плохо и работать с этим не надо
    # нормальные значения это 0.хх, 0.хх и 0.хх и т.д 
    # нам нужен вариант Х из [Х > 0 and X < 1]
    
   
    
    
    if len(math_roll.Models.value_counts()) >0:  
    
        # сначала получаем минимальное значение срне квадратичной ошибки.
        # при 1,1,1 минимальное значение будет 1,1,1 - т.е плохим
        # поэтому мы не работаем с 1,1,1 и едем в else
    
        best_math = math_roll.Models.min()
        math_roll_period = math_roll[math_roll['Models']==best_math]
        
        # math_roll_period - может содержать более одного варианта, в этом случае мы берём наибольший период.    
        # выбор происходит с помощью обработки исключений, для len(math_roll_period.Periods) == 1 
        # int(math_roll_period.Periods) допустим, для len>1 возбуждает ислючение
        # в результате получаем anime == int
        
        try:
            anime = int(math_roll_period.Periods)
        except TypeError:
            anime = int(math_roll_period.Periods.max())
            
        # в len(math_roll_period.Periods) > 1 типов данных также может быть несколько
        # в этом случае мы берём период по значение int(anime) которое было полученно ранее.    
            
        try:
            int(math_roll_period.Types) 
        except TypeError:
            math_roll_period = math_roll_period[math_roll_period['Periods']==anime] 
            
        #  опередляемся с данными, которые подаём в модель, тип 1 - простоые, тип -2 - логарифмированные
    
        if math_roll_period.Types.values == 1:
            data = test_math.math
        else:
            data = test_math_log.log_value  
            
        # запускаем модель указав глубину предсказания
        
        n_preds_math = 3 # грубина предсказания
    
        model = HoltWinters(data[-anime:]       # длинна выборки с конца
                        , slen = 2,             # длинна сезона
                        alpha = best_math[0],   # известно
                        beta = best_math[1],    # известно
                        gamma = best_math[2],   # известно
                        n_preds = n_preds_math, # глубина предсказания
                        scaling_factor = 2.56)  # 2.56 по учебнику
        model.triple_exponential_smoothing()    # execute!
    
        math = model # сохраняем модель предсказания дат поломок
        
        # формируем вывод
        
        math_result = math.result    # да я люблю сохранять явно
        
        result_math = math_result[-n_preds_math:] # даты надо рабатат ьс нимим! 
        
        # прогнозы: 0 = глубина 1, 1 = глубина 2, 2 = глубина 3, может принимать отрицательные значения
        # отрицательные значеня по умолчанию считаються периодом отказаустойчиваости ТС 
        # период отказа устойчивости это наша допустима погрешность счисления
        
        math_time = {"time":[result_math[0],result_math[1],result_math[2]]}
        
        math_time = pd.DataFrame(math_time)
        
        # result_list возможно нужно перенести ниже, пока оставлю здесь мы с него будем лепить вывод  
        
        result_list = df.tail(1)     
        
        # календарная функция дат, даёт 3 варианта будущего
        
        
        def prognoz(x_math):
            result_list['Календарь'] = result_list.index
            start_date = pd.to_datetime(result_list['Календарь'].values)
            end_date_math = timedelta(x_math)
            result_date_math = start_date + end_date_math
            return result_date_math
        
        try:
            if math_time.time[0] > 0:
                future_1 = prognoz(math_time.time[0])
                future_1 = future_1.strftime('%Y-%m-%d')
            else:
                future_1 = int(math_time.time[0])
                future_1 = str(future_1)
                future_1 = future_1.replace('-','Стойкость ')
        except Exception as e:
            if math_time.time[0] > 0:
                future_1 = prognoz(math_time.time[0])
                future_1 = future_1.strftime('%Y-%m-%d')
            else:
                future_1 = str(math_time.time[0])
                future_1 = str(future_1)
                future_1 = future_1.replace('-','Стойкость ')
            
        try:    
            if math_time.time[1] > 0:
                start_date = prognoz(math_time.time[0])
                end_date = timedelta(math_time.time[1])
                future_2 = start_date +  end_date
                future_2 = future_2.strftime('%Y-%m-%d')
            else:
                future_2 = str(math_time.time[1])
                future_2 = str(future_2)
                future_2 = future_2.replace('-','Стойкость ')
        except Exception as e:
            if math_time.time[1] > 0:
                start_date = prognoz(math_time.time[0])
                end_date = timedelta(math_time.time[1])
                future_2 = start_date +  end_date
                future_2 = future_2.strftime('%Y-%m-%d')
            else:
                future_2 = str(math_time.time[1])
                future_2 = str(future_2)
                future_2 = future_2.replace('-','Стойкость ')
        
        
        try:
            if math_time.time[2] > 0:
                start_date = prognoz(math_time.time[0])
                end_date = timedelta(math_time.time[1])
                last_date = timedelta(math_time.time[2])        
                future_3 = start_date +  end_date + last_date
                future_3 = future_3.strftime('%Y-%m-%d') 
            else:
                future_3 = int(math_time.time[2]) 
                future_3 = str(future_3)
                future_3 = future_3.replace('-','Стойкость ')
        except Exception as e:
            if math_time.time[2] > 0:
                start_date = prognoz(math_time.time[0])
                end_date = timedelta(math_time.time[1])
                last_date = timedelta(math_time.time[2])        
                future_3 = start_date +  end_date + last_date
                future_3 = future_3.strftime('%Y-%m-%d') 
            else:
                future_3 = str(math_time.time[2]) 
                future_3 = str(future_3)
                future_3 = future_3.replace('-','Стойкость ')
            
        
        result_list['wait_sum'] = result_list.простой.copy()
        result_list.wait_sum = result_list.wait_sum.dt.days.astype('int16') 
        result_list.index = result_list._ГарНомер
        result_list.rename(columns={'_Дата': 'Посл_ремонт','wait_sum':'простой_общ'}, inplace=True)
        result_list = result_list.drop(['_ГарНомер','delta','range','math','wait','простой'], axis=1)
        
        # не буду пока им показывать как чиняться машины по будням и выходным.
        
        result = result_list.copy()
        result = result.drop(['weekday','is_weekend'], axis=1)
        result['Прогноз_1'] = future_1
        result['Прогноз_2'] = future_2
        result['Прогноз_3'] = future_3  
        
    else:
        
        result_list = df.tail(1)    
        result_list['wait_sum'] = result_list.простой.copy()
        result_list.wait_sum = result_list.wait_sum.dt.days.astype('int16') 
        result_list.index = result_list._ГарНомер
        result_list.rename(columns={'_Дата': 'Посл_ремонт','wait_sum':'простой_общ'}, inplace=True)
        result_list = result_list.drop(['_ГарНомер','delta','range','math','wait','простой'], axis=1)
        
        future_1 = "Ошибка"
        future_2 = "Ошибка"
        future_3 = "Ошибка"
        
        result = result_list.copy()
        result = result.drop(['weekday','is_weekend'], axis=1)
        result['Прогноз_1'] = future_1
        result['Прогноз_2'] = future_2
        result['Прогноз_3'] = future_3   
        
    try:
        pd.to_datetime(result['Прогноз_1'], format='%Y-%M-%d', errors='raise')    
        result['Поломка_1'] = 1
    except ValueError:
        result['Поломка_1'] = 0
    
    try:
        pd.to_datetime(result['Прогноз_2'], format='%Y-%M-%d', errors='raise')    
        result['Поломка_2'] = 1
    except ValueError:
        result['Поломка_2'] = 0
    
    try:
        pd.to_datetime(result['Прогноз_3'], format='%Y-%M-%d', errors='raise')    
        result['Поломка_3'] = 1
    except ValueError:
        result['Поломка_3'] = 0    
    
    try:
        result['Поломки_прогноз'] = result['Поломка_1']+ result['Поломка_2']+ result['Поломка_3']        
    except ValueError:
        result['Поломки_прогноз'] = 0                   
        
    
         
    if len(wait_roll.Models.value_counts()) >0:  
    
        best_wait = wait_roll.Models.min()
        wait_roll_period = wait_roll[wait_roll['Models']==best_wait]        
            
        try:
            manga = int(wait_roll_period.Periods)        
        except TypeError:
            manga = int(wait_roll_period.Periods.max())         
                
        try:
            int(wait_roll_period.Types)         
        except TypeError:
            wait_roll_period = wait_roll_period[wait_roll_period['Periods']==manga]
            
        # у меня было плохое настроение, поэтому в случае абсолютно одинаковых моделей выбираем нелогорафмированную    
            
        def no_classes():
            if wait_roll_period.Types.values == 1:   
                data = test_wait.wait
            else:
                data = test_wait_log.log_value  
            return data

        try:
            data = no_classes()
        except ValueError:
            data = test_wait.wait   
        
        n_preds_wait = 3 # грубина предсказания
    
        model = HoltWinters(data[-manga:]       # длинна выборки с конца
                        , slen = 2,             # длинна сезона
                        alpha = best_wait[0],   # известно
                        beta = best_wait[1],    # известно
                        gamma = best_wait[2],   # известно
                        n_preds = n_preds_wait, # глубина предсказания
                        scaling_factor = 2.56)  # 2.56 по учебнику
        model.triple_exponential_smoothing()    # execute!
        
        # нужно подавить научное представление в wait и возможно в math
        
        wait = model # модель предсказания для простоя
            
        wait_result = wait.result
            
        # результаты предсказаний
                
        result_wait = wait_result[-n_preds_wait:] # простой  
        
        wait_time = {"time":[result_wait[0],result_wait[1],result_wait[2]]}
        
        wait_time = pd.DataFrame(wait_time)
                
        result_list = df.tail(1)        
        
        if wait_time.time[0] > 0:
            future_1 = int(wait_time.time[0])        
        else:
            future_1 = int(wait_time.time[0])
            future_1 = str(future_1)
            future_1 = future_1.replace('-','Стойкость ')
            
        if wait_time.time[1] > 0:
            future_2 = int(wait_time.time[1])        
        else:
            future_2 = int(wait_time.time[1])
            future_2 = str(future_2)
            future_2 = future_2.replace('-','Стойкость ')
            
        if wait_time.time[2] > 0:
            future_3 = int(wait_time.time[2])          
        else:
            future_3 = int(wait_time.time[2]) 
            future_3 = str(future_3)
            future_3 = future_3.replace('-','Стойкость ')
            
        result_list['wait_sum'] = result_list.простой.copy()
        result_list.wait_sum = result_list.wait_sum.dt.days.astype('int16') 
        result_list.index = result_list._ГарНомер
        result_list.rename(columns={'_Дата': 'Посл_ремонт','wait_sum':'простой_общ'}, inplace=True)
        result_list = result_list.drop(['_ГарНомер','delta','range','math','wait','простой'], axis=1)
        
        # не буду пока им показывать как чиняться машины по будням и выходным.
        
        result_w = result_list.copy()
        result_w = result_w.drop(['weekday','is_weekend'], axis=1)
        result_w['Прогноз_1'] = future_1
        result_w['Прогноз_2'] = future_2
        result_w['Прогноз_3'] = future_3 
             
    else:
        result_list = df.tail(1)    
        result_list['wait_sum'] = result_list.простой.copy()
        result_list.wait_sum = result_list.wait_sum.dt.days.astype('int16') 
        result_list.index = result_list._ГарНомер
        result_list.rename(columns={'_Дата': 'Посл_ремонт','wait_sum':'простой_общ'}, inplace=True)
        result_list = result_list.drop(['_ГарНомер','delta','range','math','wait','простой'], axis=1)
        
        future_1 = "Ошибка"
        future_2 = "Ошибка"
        future_3 = "Ошибка"
        
        result_w = result_list.copy()
        result_w = result_w.drop(['weekday','is_weekend'], axis=1)
        result_w['Прогноз_1'] = future_1
        result_w['Прогноз_2'] = future_2
        result_w['Прогноз_3'] = future_3 
        
    try:
        result_w['Прогноз_1'] = result_w['Прогноз_1'].astype('int64')
        result_w['Сумма_1'] = result_w['Прогноз_1']    
    except ValueError:
        result_w['Сумма_1'] = 0
    
    try:
        result_w['Прогноз_2'] = result_w['Прогноз_2'].astype('int64')
        result_w['Сумма_2'] = result_w['Прогноз_2']  
    except ValueError:
        result_w['Сумма_2'] = 0
    
    try:
        result_w['Прогноз_3'] = result_w['Прогноз_3'].astype('int64')
        result_w['Сумма_3'] = result_w['Прогноз_3'] 
    except ValueError:
        result_w['Сумма_3'] = 0    
    
    try:
        result_w['Простой_прогноз'] = result_w['Сумма_1']+ result_w['Сумма_2']+ result_w['Сумма_3']        
    except ValueError:
        result_w['Простой_прогноз'] = 0  
    
    try:
        result_w['Простой_сумма'] = result_w['Простой_прогноз']+ result_w['простой_общ']        
    except ValueError:
        result_w['Простой_сумма'] = 0 
    
    
    # заполняем пропуски в верхней строке простоя в таблице поомок
    
    result['Простой_прогноз'] = result_w['Простой_прогноз']
    result['Простой_сумма'] = result_w['Простой_сумма']
    
    result['Сумма_1'] = result_w['Сумма_1']
    result['Сумма_2'] = result_w['Сумма_2']
    result['Сумма_3'] = result_w['Сумма_3']
    
    # 24_11_2020 
    
    result['Прогноз_общий'] = result['Поломки_прогноз']
    result_w['Прогноз_общий'] = result_w['Простой_прогноз']
    
    # коннетируем таблицы    
    
    total = pd.concat([result, result_w])   
            
    if 'Календарь' in total.columns:
        total = total.drop(['Календарь'], axis=1) 
    
    anime = total
        
    # апдейт№1 22_10_2020

    def anime_time():
        anime_1 = anime.head(1)
        anime_1 = anime_1.drop(['Поломка_2','Поломка_3','Поломки_прогноз',
                      'Прогноз_2','Прогноз_3','Простой_сумма',
                     'Простой_прогноз','Сумма_2','Сумма_3'],axis=1)

        anime_2 = anime.head(1)
        anime_2 = anime_2.drop(['Поломка_1','Поломка_3','Поломки_прогноз',
                      'Прогноз_1','Прогноз_3','Простой_сумма',
                     'Простой_прогноз','Сумма_1','Сумма_3'],axis=1)


        anime_3 = anime.head(1)
        anime_3 = anime_3.drop(['Поломка_2','Поломка_1','Поломки_прогноз',
                      'Прогноз_2','Прогноз_1','Простой_сумма',
                     'Простой_прогноз','Сумма_2','Сумма_1'],axis=1)

        anime_1.rename(columns={'Поломка_1': 'Поломка', 'Прогноз_1': 'Прогноз','Сумма_1':'Сумма'}, inplace=True)        
        anime_2.rename(columns={'Поломка_2': 'Поломка', 'Прогноз_2': 'Прогноз','Сумма_2':'Сумма'}, inplace=True)     
        anime_3.rename(columns={'Поломка_3': 'Поломка', 'Прогноз_3': 'Прогноз','Сумма_3':'Сумма'}, inplace=True) 

        anime_1.index = anime_1.Прогноз
        anime_2.index = anime_2.Прогноз
        anime_3.index = anime_3.Прогноз

        total = pd.concat([anime_1, anime_2,anime_3]) 
    
        return total

    two_time = anime_time()

    def clock(df,s_year,s_month,s_day,e_year,e_month,e_day):
        df['_Дата'] = pd.to_datetime(df['Прогноз'], errors='coerce') 
        start = datetime.datetime( s_year,s_month,s_day)
        end = datetime.datetime(e_year,e_month,e_day)
        mask = (df['_Дата'] > start) & (df['_Дата'] <= end)
        result = df.loc[mask] 
        return result
    
    # здесь задаём диапазон поломк на месяц и два месяца вперёд

    time_1 = clock(two_time, 2020,12,1,2020,12,31)

    time_2 = clock(two_time, 2020,1,1,2020,1,31)
    
    try:
        def time_result(df):
            event = df.Поломка.sum()
            event_time = df.Сумма.sum()
            df = df.head(1)
            event_total_time = event_time + int(df.простой_общ.values) # засбоило - онли ван лен аррайк куд бы конвертед
            return event,event_time,event_total_time

        ad_1,ad_2,ad_3 = time_result(time_1)
    except TypeError:
        ad_1,ad_2 = 0,0
        ad_3 = anime.простой_общ.values[0]
    
    def far_result(df,ad_3):
        event = df.Поломка.sum()
        event_time = df.Сумма.sum()    
        event_total_time = event_time + ad_3
        return event,event_time,event_total_time

    soul_1,soul_2,soul_3 = far_result(time_2,ad_3)   

 
    roll = {'Поломок_мес':[ad_1],
            'Простоя_мес':[ad_2],
                'Общего_простоя_мес':[ad_3],
        'Поломок_мес_2':[soul_1],
            'Простоя_мес_2':[soul_2],
                'Общего_простоя_мес_2':[soul_3]
       }
    
    roll = pd.DataFrame(roll)
    roll.index = anime.Номер.head(1)
    
    # перенесено и включено заново 24_11_2020
        
    total = total.drop(['Сумма_1','Сумма_2','Сумма_3','Поломка_1','Поломка_2','Поломка_3','Простой_прогноз','Поломки_прогноз' ],axis=1)
    
        
    return (total,roll)

In [ ]:
def anime_life(df_n):    
    one = pd.DataFrame()
    for x in df_n._ГарНомер.unique():
        tablet = anime_holt(x,df_n)
        vision = pd.concat([one,tablet[0]])             
        one = vision.copy() 
    return vision

In [ ]:
def sad_life(df_n):    
    one = pd.DataFrame()
    for x in df_n._ГарНомер.unique():
        tablet = anime_holt(x,df_n)
        vision = pd.concat([one,tablet[1]])               
        one = vision.copy() 
    return vision

In [ ]:
def anime_soul(): 
    df_n = eva_01()
    
    utt = anime_life(df_n[0]) 
    nst = anime_life(df_n[1])
    trans = anime_life(df_n[2])
    nvds = anime_life(df_n[3])
    cher = anime_life(df_n[4])
    mtc = anime_life(df_n[5])
    mtk = anime_life(df_n[6])
    sam = anime_life(df_n[7])
    
    utt.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/utt_predict.csv', 
                  encoding='utf-8',index=True)
    
    nst.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nst_predict.csv', 
                  encoding='utf-8',index=True)
    trans.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/trans_predict.csv', 
                  encoding='utf-8',index=True)
    nvds.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nvds_predict.csv', 
                  encoding='utf-8',index=True)
    cher.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/cher_predict.csv', 
                  encoding='utf-8',index=True)
    mtc.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtc_predict.csv', 
                  encoding='utf-8',index=True)
    mtk.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtk_predict.csv', 
                  encoding='utf-8',index=True)
    sam.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/sam_predict.csv', 
                  encoding='utf-8',index=True)
    
    utt.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/utt/utt_predict.html')
    
    nst.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nst/nst_predict.html')
    trans.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/trans/trans_predict.html')
    nvds.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nvds/nvds_predict.html')
    cher.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/cher/cher_predict.html')
    mtc.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtc/mtc_predict.html')
    mtk.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtk/mtk_predict.html')
    sam.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/sam/sam_predict.html')
    
    # return (utt,nst,trans,nvds,cher,mtc,mtk,sam)  
    
    utt_sad = sad_life(df_n[0]) # и далее сохраняем вывод
    nst_sad = sad_life(df_n[1]) # и далее сохраняем вывод
    trans_sad = sad_life(df_n[2]) # и далее сохраняем вывод
    nvds_sad = sad_life(df_n[3]) # и далее сохраняем вывод
    cher_sad = sad_life(df_n[4]) # и далее сохраняем вывод
    mtc_sad = sad_life(df_n[5]) # и далее сохраняем вывод
    mtk_sad = sad_life(df_n[6]) # и далее сохраняем вывод
    sam_sad = sad_life(df_n[7]) # и далее сохраняем вывод
    
    utt_sad.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/utt/utt_predict_sad.csv', 
                  encoding='utf-8',index=True)
    
    nst_sad.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nst/nst_predict_sad.csv', 
                  encoding='utf-8',index=True)
    trans_sad.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/trans/trans_predict_sad.csv', 
                  encoding='utf-8',index=True)
    nvds_sad.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nvds/nvds_predict_sad.csv', 
                  encoding='utf-8',index=True)
    cher_sad.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/cher/cher_predict_sad.csv', 
                  encoding='utf-8',index=True)
    mtc_sad.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtc/mtc_predict_sad.csv', 
                  encoding='utf-8',index=True)
    mtk_sad.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtk/mtk_predict_sad.csv', 
                  encoding='utf-8',index=True)
    sam_sad.to_csv('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/sam/sam_predict_sad.csv', 
                  encoding='utf-8',index=True)
    
    utt_sad.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/utt/utt_predict_sad.html')
    
    nst_sad.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nst/nst_predict_sad.html')
    trans_sad.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/trans/trans_predict_sad.html')
    nvds_sad.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nvds/nvds_predict_sad.html')
    cher_sad.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/cher/cher_predict_sad.html')
    mtc_sad.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtc/mtc_predict_sad.html')
    mtk_sad.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtk/mtk_predict_sad.html')
    sam_sad.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/sam/sam_predict_sad.html')
    
    # нужно переписать suicide
    
    def suicide_my_life():

        def result(name,future):            
            score = ['Поломки_прогноз_1', 'Поломки_прогноз_2','Простой_прогноз_1',
            'Простой_прогноз_2','Общего_простоя_мес','Общего_простоя_мес_2']
            series = {name: [future.Поломок_мес.sum(),
                     future.Поломок_мес_2.sum(),
                     future.Простоя_мес.sum(),
                     future.Простоя_мес_2.sum(),
                     future.Общего_простоя_мес.sum(),
                     future.Общего_простоя_мес_2.sum()]} # засбоило здесь
            score = pd.DataFrame(index = score,data=series) 
            return score 
    
        utt_suicide = result('утт',utt_sad)
        nst_suicide = result('nst',nst_sad)
        trans_suicide = result('trans',trans_sad)
        nvds_suicide = result('nvds',nvds_sad)
        cher_suicide = result('cher',cher_sad)
        mtc_suicide = result('mtc',mtc_sad)
        mtk_suicide = result('mtk',mtk_sad)
        sam_suicide = result('sam',sam_sad)
    
        utt_suicide.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/utt/utt_predict_sum.html')    
        nst_suicide.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nst/nst_predict_sum.html')
        trans_suicide.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/trans/trans_predict_sum.html')
        nvds_suicide.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nvds/nvds_predict_sum.html')
        cher_suicide.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/cher/cher_predict_sum.html')
        mtc_suicide.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtc/mtc_predict_sum.html')
        mtk_suicide.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtk/mtk_predict_sum.html')
        sam_suicide.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/sam/sam_predict_sum.html')
    
        return utt_suicide ,nst_suicide,trans_suicide,nvds_suicide,cher_suicide,mtc_suicide,mtk_suicide,sam_suicide 
    
    utt_suicide,nst_suicide,trans_suicide,nvds_suicide,cher_suicide,mtc_suicide,mtk_suicide,sam_suicide  = suicide_my_life()
    
    dfs = [utt_suicide,nst_suicide,trans_suicide,nvds_suicide,cher_suicide,mtc_suicide,mtk_suicide,sam_suicide]
    
    result_suicide = pd.concat(dfs,axis=1)
    
    # я не помню зачем, но когда то давно я тут написал какойто tablet score
    # return (utt,nst,trans,nvds,cher,mtc,mtk,sam),(utt_sad,nst_sad,trans_sad,nvds_sad,cher_sad,mtc_sad,mtk_sad,sam_sad),(tablet_score,result_suicide) 
    
    return (utt,nst,trans,nvds,cher,mtc,mtk,sam),(utt_sad,nst_sad,trans_sad,nvds_sad,cher_sad,mtc_sad,mtk_sad,sam_sad),(result_suicide)  

In [ ]:
# 27_11_2020

def night_work():
    future,suicide,die = anime_soul()
    
    def night_mod(df_2):
        mask = (df_2.Поломок_мес >0) |  (df_2.Поломок_мес_2 >0)                                              
        result = df_2[mask]
        return result
    
    # возможно нужно добавить колонку Номер в вывод night_mod 
    
    utt_res = night_mod(suicide[0])
    nst_res = night_mod(suicide[1])
    trans_res = night_mod(suicide[2])
    nvds_res = night_mod(suicide[3])
    cher_res = night_mod(suicide[4])
    mtc_res = night_mod(suicide[5])
    mtk_res = night_mod(suicide[6])
    sam_res = night_mod(suicide[7])
    
    total_utt = utt_res.merge(future[0], on=["Номер"])
    total_nst = nst_res.merge(future[1], on=["Номер"])
    total_trans = trans_res.merge(future[2], on=["Номер"])
    total_nvds = nvds_res.merge(future[3], on=["Номер"])
    total_cher = cher_res.merge(future[4], on=["Номер"])
    total_mtc = mtc_res.merge(future[5], on=["Номер"])
    total_mtk = mtk_res.merge(future[6], on=["Номер"])
    total_sam = sam_res.merge(future[7], on=["Номер"])
    
    total_utt.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/utt/utt_обзор.html')
    total_nst.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nst/nst_обзор.html')
    total_trans.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/trans/trans_обзор.html')
    total_nvds.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/nvds/nvds_обзор.html')
    total_cher.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/cher/cher_обзор.html')
    total_mtc.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtc/mtc_обзор.html')
    total_mtk.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/mtk/mtk_обзор.html')
    total_sam.to_html('C:/Users/timofeevab.URALS-CAPITAL/Downloads/Data/eva_01/sam/sam_обзор.html')    
    
    return future,suicide,die,(total_utt,total_nst,total_trans,total_nvds,total_cher,total_mtc,total_mtk,total_sam)

In [ ]:
future,suicide,die = anime_soul()